In [5]:
#A Really Dumb Spam Filter
''' Bayes's theorem'''

'''
    P("the message is spam" S / "The message contains the word bitcoin" B) = [P(B|S)P(S)] / [P(B|S)P(S) + P(B|-S)P(-S)] 
    (-S = message is not spam)
'''
'''
    For example, if 50% of spam messages have the word bitcoin, but only 1% of nonspam messages do, then the probability that any given bitcoin-containing email is spam is:        0.5/(0.5 + 0.01) = 98%
'''


'\n    For example, if 50% of spam messages have the word bitcoin, but only 1% of nonspam messages do, then the probability that any given bitcoin-containing email is spam is:        0.5/(0.5 + 0.01) = 98%\n'

In [1]:
#A More Sophisticated Spam Filter
'''
    Vocabulary of many words w1, ..., wn
    Xi for the event "a message contains the word wi".     '
    Imagine we've come up with an estimate P(Xi|S) for the probability that a spam message gonatins the ith word, and a similar estimate P(Xi|-S) for the probability that a nonspam message contains the ith word.

    The key to Naive Bayes is making the (big) assumption that the presence (or absences) of each word are independent of one another, conditional on a message being spam or not.
    This assumption means that knowing whether a certain spam message contains the word bitcoin gives you no information about whether that same message ontains the word rolex. In math terms, this means that:
        
        P(X1 = x1, ..., Xn = xn|S) = P(X1 = x1|S) * ... * P(Xn=xn|S)
    
    This is an extreme assumption. (There's a reason the technique has naive in its name.) Imagine that our vocabulary consists only of the words bitcoin and rolex, and that half of all spam messages are for "earn bitcoin" and that the other half are for "authentic rolex." In this case, the Naive Bayes estimate that a spam message contains both bitcoin and rolex is:

        P(X1 = 1, X2 = 1|S) = P(X1= 1|S)P(X2 = 1|S) = .5 * .5 = .25

    since we've assumed away the knowledge that bitcoin and rolex actually never occure together. Dispite the unrealisticness of this assumption, this model often performs well and has historically been used in actual spam filters.
'''

'\n    Vocabulary of many words w1, ..., wn\n    Xi for the event "a message contains the word wi".     \'\n    Imagine we\'ve come up with an estimate P(Xi|S) for the probability that a spam message gonatins the ith word, and a similar estimate P(Xi|-S) for the probability that a nonspam message contains the ith word.\n\n    The key to Naive Bayes is making the (big) assumption that the presence (or absences) of each word are independent of one another, conditional on a message being spam or not.\n    This assumption means that knowing whether a certain spam message contains the word bitcoin gives you no information about whether that same message ontains the word rolex. In math terms, this means that:\n        \n        P(X1 = x1, ..., Xn = xn|S) = P(X1 = x1|S) * ... * P(Xn=xn|S)\n    \n    This is an extreme assumption. (There\'s a reason the technique has naive in its name.) Imagine that our vocabulary consists only of the words bitcoin and rolex, and that half of all spam messages

In [3]:
#Implementation (STUDY THIS AND UNDERSTAND COMPLETELY)
'''
    Let's create a simple function to tokeize messages into distinct words. We'll first convert each message to lowercase, then use re.findall to extract "words" consisting of letters, numbers, and apostrophes. Finally, we'll use set to get just the distinct words
'''

from typing import Set
import re

def tokenize(text: str) -> Set[str]:
    text = text.lower()                             #Convert to lowercase
    all_words = re.findall("[a-z0-9']+", text)      #extract the words, and
    return set(all_words)                           #remove duplicates

assert tokenize("Data Science is science") == {"data", 'science', 'is'}

#We'll also define a type for our training data

from typing import NamedTuple

class Message(NamedTuple):
    text: str
    is_spam: bool

'''
    As our classifier needs to keep track of tokens, counts, and labels from the training data, we'll make it a class. Following convention, we refer to nonspam emails as ham emails
    The constructor will take just one parameter, the pseudocount to use when computing probabilities. It also initializes an empty set of tokens, counters to track how often each token is seen in spam messages and ham messages, and counts of how many spam and ham messages it has trained on:
'''

from typing import List, Tuple, Dict, Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k #smoothing factor
        
        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0

'''
    Next, we'll give it a method to train it on a bunch of messages. First, we increment the spam_messages and ham_messages counts. Then we tokenize each message text, and for each token we increment the token_spam_counts or token_ham_counts bassed on the message type:
'''

def train(self, messages: Iterable[Message]) -> None:
    for message in messages:
        #Increment message counts
        if message.is_spam:
            self.spam_messages += 1
        else:
            self.ham_messages += 1

        #Increment word counts
        for token in tokenize(message.text):
            self.tokens.add(token)
            if message.is_spam:
                self.token_spam_counts[token] += 1
            else:
                self.token_ham_counts[token] += 1

'''
    Ultimately we'll want to predict P(spam | token). As we saw earlier,  to apply Bayes's theorem we need to know P(token | spam) and P(token | ham) for each token in the vocabulary. So we'll create a "private" helper function to compute those:
'''

def _probabilities(self, token: str) -> Tuple[float, float]:
    """returns P(token | spam) and P(token | ham)"""
    spam = self.token_spam_counts[token]
    ham = self.token_ham_counts[token]

    p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
    p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)

    return p_token_spam, p_token_ham

'''
    Finally, we're ready to write our predict method. As mentioned earlier, rather than multiplying together lots of small probabilities, we'll instead sum up the log probabilities:
'''

def predict(self, text:str) -> float:
    text_tokens = tokenize(text)
    log_prob_if_spam = log_prob_if_ham = 0.0

    #Iterate through each word in our vocabulary
    for token in self.tokens:
        prob_if_spam, prob_if_ham = self._probabilities(token)

        #If *token* appears in the message,
        #add the log probability of seeing it
        if token in text_tokens:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_ham += math.log(prob_if_ham)

        #Otherwise add the log probability of _not_ seeing it,
        #which is log(1-probability of seeing it)
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_ham += math.log(1.0 - prob_if_ham)
        
        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)

#And now we have a classifier

In [4]:
#Testing our Model